In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import os
import gc
import math
import random
import pickle
import pandas as pd
import numpy as np
import multiprocessing
from tqdm.auto import tqdm

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from transformers import get_cosine_schedule_with_warmup

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset, sampler

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
random_seed = 41

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed) 
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    os.environ["PYTHONHASHSEED"] = str(seed)
    
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_everything(seed=random_seed) # Seed 고정

In [4]:
train = pd.read_csv("./data/df_train.csv")
test = pd.read_csv("./data/df_test.csv")
train.shape, test.shape

((262, 48), (175, 47))

In [5]:
y = train['class'].values
X = train.drop(['id', 'class'], axis=1).to_numpy()
X_test = test.drop(['id'], axis=1).to_numpy()

In [6]:
total = np.concatenate([X, X_test], axis=0)
total.shape

(437, 46)

In [7]:
scaler = StandardScaler()
total = scaler.fit_transform(total)
total = np.expand_dims(total, axis=1)
total.shape

(437, 1, 46)

In [8]:
class Encoder(nn.Module):
    def __init__(self, n_features, latent_dim):
        super().__init__()
        
        self.lstm0 = nn.LSTM(
            input_size=n_features,
            hidden_size=latent_dim*3,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.lstm1 = nn.LSTM(
            input_size=latent_dim*3,
            hidden_size=latent_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )

    def forward(self, x):
        x, (_, _) = self.lstm0(x)
        x, (h_n, _) = self.lstm1(x)
        
        return h_n.permute(1, 0, 2)


class Decoder(nn.Module):
    def __init__(self, n_features, latent_dim):
        super().__init__()
        self.latent_dim = latent_dim
        
        self.lstm0 = nn.LSTM(
            input_size=latent_dim,
            hidden_size=latent_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.lstm1 = nn.LSTM(
            input_size=latent_dim,
            hidden_size=latent_dim*3,
            num_layers=1,
            batch_first=True,
            bidirectional=False
        )
        self.linear = nn.Linear(in_features=latent_dim*3, out_features=n_features)

    def forward(self, x):
        x = x.repeat(1, 31, 1)
        
        x, (_, _) = self.lstm0(x)
        x, (_, _) = self.lstm1(x)
        
        x = self.linear(x)

        return x


class AutoEncoder(nn.Module):
    def __init__(self, n_features=46, latent_dim=4, device=None):
        super().__init__()
        
        self.encoder = Encoder(n_features, latent_dim).to(device)
        self.decoder = Decoder(n_features, latent_dim).to(device)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        
        return x

In [9]:
CFG = {
    'EPOCHS':10000,
    'LEARNING_RATE':0.0031,
    'BATCH_SIZE':437,
}

In [10]:
def validation(model, criterion, test_loader, device):
    model.eval()
    
    model_preds = []
    true_labels = []
    
    val_loss = []

    with torch.no_grad():
        for x, label in iter(test_loader):
            x, label = x.to(device), label.to(device)

            model_pred = model(x)

            loss = criterion(model_pred, label)

            val_loss.append(loss.item())

    return np.mean(val_loss)

In [ ]:
# DataLoader 정의
train_dataset = TensorDataset(torch.from_numpy(total).type(torch.float), torch.from_numpy(total).type(torch.float))
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=2, worker_init_fn=seed_worker)

# 학습 모델 설정
model = AutoEncoder().to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])  # Adam
scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=len(train_loader) * int(CFG['EPOCHS']*0.1),
    num_training_steps=len(train_loader) * CFG['EPOCHS']
)

best_score = 100

# train
for epoch in range(1,CFG['EPOCHS']+1):
    model.train()
    train_loss = []
    for x, label in iter(train_loader):            
        x, label = x.to(device), label.to(device)

        optimizer.zero_grad()

        model_pred = model(x)

        loss = criterion(model_pred, label)

        loss.backward()
        optimizer.step()
        scheduler.step()

        train_loss.append(loss.item())

    tr_loss = np.mean(train_loss)

    print(f'Epoch [{epoch}], Train Loss : [{tr_loss:.5f}]')

    if best_score > tr_loss:
        torch.save(model.state_dict(), f'./models/AutoEncoder_total.pt')
        best_score = tr_loss

Epoch [1], Train Loss : [1.03530]
Epoch [2], Train Loss : [1.03530]
Epoch [3], Train Loss : [1.03530]
Epoch [4], Train Loss : [1.03530]
Epoch [5], Train Loss : [1.03529]
Epoch [6], Train Loss : [1.03528]
Epoch [7], Train Loss : [1.03527]
Epoch [8], Train Loss : [1.03525]
Epoch [9], Train Loss : [1.03523]
Epoch [10], Train Loss : [1.03521]
Epoch [11], Train Loss : [1.03519]
Epoch [12], Train Loss : [1.03517]
Epoch [13], Train Loss : [1.03514]
Epoch [14], Train Loss : [1.03511]
Epoch [15], Train Loss : [1.03508]
Epoch [16], Train Loss : [1.03504]
Epoch [17], Train Loss : [1.03501]
Epoch [18], Train Loss : [1.03497]
Epoch [19], Train Loss : [1.03493]
Epoch [20], Train Loss : [1.03488]
Epoch [21], Train Loss : [1.03484]
Epoch [22], Train Loss : [1.03479]
Epoch [23], Train Loss : [1.03474]
Epoch [24], Train Loss : [1.03468]
Epoch [25], Train Loss : [1.03463]
Epoch [26], Train Loss : [1.03457]
Epoch [27], Train Loss : [1.03451]
Epoch [28], Train Loss : [1.03445]
Epoch [29], Train Loss : [1.0